In [1]:
import mne
from netCDF4 import Dataset
import json
import dask.array as da
from dask import delayed
import os, logging
from services.utils.timing import TimingContext
import pyarrow as pa
from services.delta_lake import Duck_Lake
from services.utils.directory_utils import get_tmpdir
from prefect import flow, task
from prefect_dask import DaskTaskRunner
from dataclasses import dataclass, asdict
from typing import List
from uuid import uuid4 as uuid

logging.basicConfig()
logging.root.setLevel(logging.INFO)

ducklake = Duck_Lake()

In [11]:
my_edf_file_path = os.path.join(
    os.environ["CONTAINER_FILE_STORAGE_PATH"],
    "test33_HypoactiveHeidi_05_DAY1_PROCESSED.edf",
)
my_parquet_output_dir = os.path.join(os.environ["CONTAINER_FILE_STORAGE_PATH"], "test")

# Non-EEG
misc_channels = [
    "pitch",
    "roll",
    "heading",
    "GyrZ",
    "MagZ",
    "Tag_On",
    "Depth",
    "MagX",
    "MagY",
]

@dataclass
class SignalSchema:
    signal_name: str
    frequency: float
    start_time: float
    data: List[float]

@task
def read_signal(
    edf_file_path,
    signal_name,
    mode: str = "SINGLE"
):
    """Function to read a single signal from an EDF file."""
    raw = mne.io.read_raw_edf(edf_file_path, include=[signal_name], preload=False)
    signal = raw.pick(signal_name).get_data()
    
    if mode == 'SINGLE':
        return SignalSchema(
            signal_name=[signal_name],
            frequency=[raw.info["sfreq"]],
            start_time=[pa.scalar(raw.info["meas_date"].timestamp(), type=pa.timestamp('us', tz="UTC"))],
            data=[signal[0]],
        )
    if mode == "TABLE":
        return SignalSchema(
            signal_name=signal_name,
            frequency=raw.info["sfreq"],
            start_time=pa.scalar(raw.info["meas_date"].timestamp(), type=pa.timestamp('us', tz="UTC")),
            data=signal[0],
        )
        


@flow
def process_edf(
    edf_file_path: str,
    schema: pa.schema
):
    with TimingContext("EDF Read"):
        buff = []
        tmpdir = get_tmpdir()
        for signal_name in ['pitch', ]:
        # for signal_name in raw.ch_names[0:2]:
            signal = read_signal(edf_file_path, signal_name)
            table = pa.table(asdict(signal), schema=schema)
            fname = os.path.join(tmpdir, f'{signal_name}.parquet')
            pa.parquet.write_table(table, fname)
            buff.append(fname)
        
        ducklake.write_parquet_to_delta(
            parquet_files=buff,
            schema=schema,
            mode="append",
            partition_by=['signal_name'],
            name="test",
            description="test"
        )


schema = pa.schema(
    [
        pa.field("signal_name", pa.string()),
        pa.field("frequency", pa.float64()),
        pa.field("start_time", pa.timestamp('us', tz="UTC")),
        pa.field("data", pa.list_(pa.float64())),
    ]
)

with TimingContext("Main"):
    process_edf(my_edf_file_path, schema)


/usr/local/lib/python3.12/site-packages/pydantic/_internal/_generate_schema.py:404: UserWarning: <cyfunction schema at 0xffff36129d80> is not a Python type (it may be an instance of an object), Pydantic will allow any object with no validation since we cannot even enforce that the input is an instance of the given type. To get rid of this error wrap the type with `pydantic.SkipValidation`.
  warn(
/usr/local/lib/python3.12/site-packages/pydantic/_internal/_fields.py:201: UserWarning: Field name "schema" in "ProcessEdf" shadows an attribute in parent "V2ValidatedFunction.create_model.<locals>.DecoratorBaseModel"
  warnings.warn(
/usr/local/lib/python3.12/site-packages/pydantic/_internal/_fields.py:201: UserWarning: Field name "schema" in "ProcessEdf" shadows an attribute in parent "V2ValidatedFunction.create_model.<locals>.DecoratorBaseModel"
  warnings.warn(
/usr/local/lib/python3.12/site-packages/pydantic/_internal/_generate_schema.py:404: UserWarning: <cyfunction schema at 0xffff3612

02:13:29.826 | INFO    | prefect.engine - Created flow run 'faithful-mammoth' for flow 'process-edf'

02:13:29.828 | INFO    | Flow run 'faithful-mammoth' - View at http://prefect:4200/flow-runs/flow-run/33561671-37e2-488a-a525-d20bf8ef8301

Extracting EDF parameters from /data/files/test33_HypoactiveHeidi_05_DAY1_PROCESSED.edf...
EDF file detected
Setting channel info structure...
Creating raw.info structure...


/tmp/ipykernel_12/1447355065.py:60: RuntimeWarning: Channels contain different highpass filters. Highest filter setting will be stored.
  raw = mne.io.read_raw_edf(edf_file_path, preload=False)
/usr/local/lib/python3.12/site-packages/mne/io/edf/edf.py:782: RuntimeWarning: All-NaN axis encountered
  value = np.nanmax([_prefilter_float(x) for x in values])
/tmp/ipykernel_12/1447355065.py:60: RuntimeWarning: Channels contain different lowpass filters. Lowest filter setting will be stored.
  raw = mne.io.read_raw_edf(edf_file_path, preload=False)
/usr/local/lib/python3.12/site-packages/mne/io/edf/edf.py:784: RuntimeWarning: All-NaN axis encountered
  value = np.nanmin([_prefilter_float(x) for x in values])


02:13:30.117 | INFO    | Flow run 'faithful-mammoth' - Created task run 'read_signal-0' for task 'read_signal'

02:13:30.119 | INFO    | Flow run 'faithful-mammoth' - Executing 'read_signal-0' immediately...

Extracting EDF parameters from /data/files/test33_HypoactiveHeidi_05_DAY1_PROCESSED.edf...
EDF file detected
Setting channel info structure...
Creating raw.info structure...


02:13:31.808 | INFO    | Task run 'read_signal-0' - Finished in state Completed()

02:13:31.963 | INFO    | Flow run 'faithful-mammoth' - Finished in state Completed('All states completed.')

In [9]:
print(os.environ["CONTAINER_DELTA_LAKE_PATH"])

ducklake.conn.execute(
    f'''
    SELECT data 
    FROM delta_scan('{os.environ["CONTAINER_DELTA_LAKE_PATH"]}') LIMIT 1
    '''
).pl()

/data/delta-lake


FloatProgress(value=0.0, layout=Layout(width='auto'), style=ProgressStyle(bar_color='black'))

data
list[f64]
"[-0.000031, -0.000033, … 0.000175]"
